In [ ]:
import scraping
from scraping import scraper
import pandas as pd
import ast

In [ ]:
topics_urls = []
base_topic_url = "https://www.webmd.com/a-to-z-guides/health-topics?pg="
for num in range(97, 97 + 26):
    topics_urls.append(base_topic_url + chr(num))

In [ ]:
'''
for i in range(len(topics_urls)):
    webscraper = scraper(topics_urls[i:i+1], "./scraped/out_"+chr(97+i)+".csv")
    webscraper.scrape()
'''

In [ ]:
df = pd.read_csv("scraped/webmd_diseases.csv")
df.loc[0, "body"]

In [ ]:
def parse(text):
    text = text[1:-1].replace("\\xa0", ' ').replace("\",", "|").replace("',", "|").replace("\\", "").split("|")
    for i in range(len(text)):
        text[i] = text[i].strip(" \"'")
    return text

In [ ]:
def df_with_map_text_chunk_to_index(df):
    mapped = pd.DataFrame(columns=["df_index", "text"])
    for i in range(df.shape[0]):
        parsed = parse(df.loc[i, "body"])
        article_row = []
        for chunk in parsed:
            new_row = {"df_index": i, "text": chunk}
            article_row.append(new_row)
        new_df = pd.DataFrame(article_row)
        mapped = pd.concat([mapped, new_df])
    return mapped

In [ ]:
p = df_with_map_text_chunk_to_index(df)
p.shape

In [ ]:
test = pd.read_csv("scraped/chunked_embedding.csv", converters={"embedding": ast.literal_eval})


In [ ]:
val = test.loc[:, 'embedding'].values
for l in val:
    print(len(l))